References : https://github.com/adlnlp/K-MHaS

# 한국어 혐오 발언 다중 레이블 분류 : K-MHaS (Korean Multi-label Hate Speech Dataset)

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


## 데이터셋 로드 및 구조 확인

In [ ]:
from datasets import load_dataset

dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 78977
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 8776
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21939
    })
})

In [ ]:
dataset = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [ ]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 21939
})

In [ ]:
dataset.features

{'text': Value(dtype='string', id=None),
 'label': Sequence(feature=ClassLabel(names=['origin', 'physical', 'politics', 'profanity', 'age', 'gender', 'race', 'religion', 'not_hate_speech'], id=None), length=-1, id=None)}

In [ ]:
print('테스트 데이터 셋의 크기 :', len(dataset['text']))
print('첫번째 샘플 출력 :', dataset['text'][0])
print('첫번째 샘플의 레이블 출력 :', dataset['label'][0])

테스트 데이터 셋의 크기 : 21939
첫번째 샘플 출력 : 그만큼 길예르모가 잘했다고 보면되겠지 기대되네 셰이프 오브 워터
첫번째 샘플의 레이블 출력 : [8]


- 데이터셋의 [깃허브](https://github.com/adlnlp/K-MHaS)로부터 확인할 수 있는 각 레이블이 의미하는 바는 다음과 같습니다. 레이블은 0부터 8까지 총 9개의 레이블이 존재합니다.

      class_label:
        names:
          0: origin (출신차별)
          1: physical (외모차별)
          2: politics (정치성향차별)
          3: profanity (혐오욕설)
          4: age (연령차별)
          5: gender (성차별)
          6: race (인종차별)
          7: religion (종교차별)
          8: not_hate_speech (혐오아님)

In [ ]:
print('두번째 샘플 출력 :', dataset['text'][1])
print('두번째 샘플의 레이블 출력 :', dataset['label'][1])

두번째 샘플 출력 : "1. 8넘의 문재앙"
두번째 샘플의 레이블 출력 : [2, 3]


## 데이터셋 전처리

In [ ]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

In [ ]:
# 훈련 데이터, 검증 데이터, 테스트 데이터를 로드합니다.

train = load_dataset("jeanlee/kmhas_korean_hate_speech", split="train")
validation = load_dataset("jeanlee/kmhas_korean_hate_speech", split="validation")
test = load_dataset("jeanlee/kmhas_korean_hate_speech", split="test")

In [ ]:
# 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다.

train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train['text']))
validation_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', validation['text']))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test['text']))

In [ ]:
# 정답인 레이블의 위치에는 1, 나머지 위치에는 0을 기록합니다.
# 레이블 전처리 예시)
# [8]    -> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0] : 의미적으로는 [no, no, no, no, no, no, no, no, no, yes]
# [2, 3] -> [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] : 의미적으로는 [no, no, yes, yes, no, no, no, no, no, no]

from sklearn.preprocessing import MultiLabelBinarizer

enc = MultiLabelBinarizer()

def multi_label(example):
    enc_label = enc.fit_transform(example['label'])
    float_arr = np.vstack(enc_label[:]).astype(float)
    update_label = float_arr.tolist()
    return update_label

train_labels = multi_label(train)
validation_labels = multi_label(validation)
test_labels = multi_label(test)

In [ ]:
test_sentences[:5]

['[CLS] 그만큼 길예르모가 잘했다고 보면되겠지 기대되네 셰이프 오브 워터 [SEP]',
 '[CLS] "1. 8넘의 문재앙" [SEP]',
 '[CLS] "문재인 정권의 내로남불은 타의 추종을 불허하네. 자한당 욕할거리도 없음." [SEP]',
 '[CLS] "짱개들 지나간 곳은 폐허된다 ㅋㅋ" [SEP]',
 '[CLS] 곱창은 자갈치~~~~~ [SEP]']

In [ ]:
# 각 레이블은 기존에 [8], [2, 3], [2], [0], [8] 이었으며 전처리 후 아래와 같이 변경됨.
test_labels[:5]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
 [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]

## BERT 토크나이저를 이용한 전처리

In [ ]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
max_len = 128

def data_to_tensor (sentences, labels, max_len):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 0으로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 0, 0, 0]
  input_ids = pad_sequences(input_ids, maxlen=max_len, dtype="long", truncating="post", padding="post")

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks

In [ ]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels, max_len)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels, max_len)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels, max_len)

In [ ]:
tokenizer.decode([2])

'[CLS]'

In [ ]:
tokenizer.decode([3])

'[SEP]'

In [ ]:
print('정수 인코딩 결과:', test_inputs[0])
print('-' * 20)
print('원본 문장 복원 결과:', tokenizer.decode(test_inputs[0]))
print('-' * 20)
print('어텐션 마스크:', test_masks[0])
print('-' * 20)
print('샘플의 길이:', len(test_inputs[0]))
print('-' * 20)
print('레이블:', test_labels[0])

정수 인코딩 결과: tensor([    2,  5569,   647,  2194,  2131,  2391,  2116,  1521,  2371,  4683,
         1160,  2460,  2496,  2918,  2118,  3869,  2496,  2203,  1280,  2052,
         2515, 10257,  9478,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,    

In [ ]:
batch_size = 64

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 78977
검증 데이터의 크기: 8776
테스트 데이터의 크기: 21939


## GPU가 정상 셋팅되었는지 확인.  
Colab에서 GPU를 사용하기 위해서는 아래와 같이 설정이 되어있어야만 합니다.  

* 런타임 > 런타임 유형 변경 > 하드웨어 가속기 > 'GPU' 선택

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


## 모델 로드하기

In [ ]:
num_labels = 9

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels, problem_type="multi_label_classification")
model.cuda()

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# 옵티마이저 선택
optimizer = AdamW(model.parameters(), lr = 2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 몇 번의 에포크(전체 데이터에 대한 학습 횟수)를 할 것인지 선택
epochs = 4

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    """
    멀티레이블 분류 문제의 예측값과 실제 라벨을 기반으로 다양한 평가 지표를 계산합니다.
    이미 확률로 변환된 예측값을 사용하여, 임계값을 기준으로 레이블을 결정하고 평가 지표를 계산합니다.

    Parameters:
    predictions (numpy.ndarray): 모델에 의해 예측된 확률값. 각 샘플에 대한 레이블 확률을 포함합니다.
    labels (numpy.ndarray): 실제 라벨값. 각 샘플에 대한 실제 레이블을 포함합니다.
    threshold (float): 레이블을 결정하기 위한 임계값. 기본값은 0.5입니다.

    Returns:
    dict: 다양한 평가 지표(metrics)에 대한 값들을 담은 사전.
    """

    # 임계값을 넘는 확률 값을 가진 경우 1로 예측했다고 간주합니다.
    y_pred = (predictions >= threshold).astype(int)

    # 사용 가능한 메트릭들을 사용합니다.
    accuracy = accuracy_score(labels, y_pred)
    f1_macro_average = f1_score(y_true=labels, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=labels, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=labels, y_pred=y_pred, average='weighted', zero_division=0)
    roc_auc = roc_auc_score(labels, y_pred, average='micro')

    # 메트릭 결과에 대해서 리턴합니다.
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average,
               'roc_auc': roc_auc}

    return metrics

## 모델 학습

In [ ]:
def train_epoch(model, train_dataloader, optimizer, device):
    """
    하나의 에포크 동안 모델을 학습시키는 함수입니다.

    Parameters:
    model (torch.nn.Module): 학습시킬 모델 객체.
    train_dataloader (torch.utils.data.DataLoader): 학습 데이터셋의 DataLoader.
    optimizer (torch.optim.Optimizer): 최적화 알고리즘을 구현하는 객체.
    device (torch.device): 학습에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 학습 손실값.
    """

    total_train_loss = 0  # 학습 손실을 누적할 변수 초기화
    model.train()  # 모델을 학습 모드로 설정

    # 학습 데이터로더를 순회하며 배치 단위로 학습
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Training Batch"):
        batch = tuple(t.to(device) for t in batch)  # DataLoader에서 배치를 받아 각 텐서를 지정된 장치로 이동
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 입력 ID, 마스크, 라벨 추출

        # 모델에 배치를 전달하여 손실값 계산
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        # 손실값 추출
        loss = outputs.loss

        optimizer.zero_grad()  # 그래디언트 초기화
        loss.backward()  # 역전파를 통해 그래디언트 계산
        optimizer.step()  # 매개변수 업데이트

        total_train_loss += loss.item()  # 총 손실에 더함

    avg_train_loss = total_train_loss / len(train_dataloader)  # 평균 학습 손실 계산

    return avg_train_loss  # 평균 학습 손실 반환

In [ ]:
def evaluate(model, validation_dataloader, device):
    """
    모델을 사용하여 검증 데이터셋에 대한 평가를 수행하는 함수입니다. 이 함수는 멀티레이블 분류 문제에 적합하게 설계되었습니다.
    각 레이블에 대한 예측을 독립적으로 수행하고, 여러 평가 지표를 계산하여 모델의 성능을 평가합니다.

    Parameters:
    model (torch.nn.Module): 평가할 모델 객체. 이 모델은 멀티레이블 분류를 위한 출력을 생성해야 합니다.
    validation_dataloader (torch.utils.data.DataLoader): 검증 데이터셋의 DataLoader.
    이 DataLoader는 입력 데이터와 정답 레이블을 배치로 제공합니다.
    device (torch.device): 평가에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 검증 손실값. 검증 데이터셋에 대한 모델의 손실을 평균낸 값입니다.
    dict: 다양한 평가 지표(metrics)에 대한 값들을 담은 사전. 이 사전은 정확도(accuracy),
    F1 점수(macro, micro, weighted) 및 ROC-AUC 점수를 포함합니다.
    """

    model.eval()  # 모델을 평가 모드로 설정

    total_eval_loss = 0
    predictions, true_labels = [], []  # 모델 예측 확률과 실제 라벨을 저장할 리스트

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)  # 배치 데이터를 디바이스로 이동
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():  # 그래디언트 계산을 수행하지 않음
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        total_eval_loss += loss.item()  # 총 손실에 더함

        logits = outputs.logits.detach().cpu().numpy()  # 모델 예측값(로짓)을 numpy 배열로 변환
        label_ids = b_labels.to('cpu').numpy()  # 실제 라벨값을 numpy 배열로 변환

        # 시그모이드 함수를 적용하여 로짓을 확률로 변환
        # 멀티레이블 분류에서는 각 레이블에 대한 독립적인 예측을 수행합니다.
        # 예를 들어, logits 값이 [-1.2, 2.3, -0.4, 1.0, -2.2, 0.5, -0.7, 1.2, 0.3]로 주어지고,
        # 각각의 레이블에 대해 시그모이드 함수를 적용하면, 확률은
        # probs = [0.23, 0.91, 0.40, 0.73, 0.10, 0.62, 0.33, 0.77, 0.57]과 같이 계산됩니다.
        # 실제 예측 = [0 1 0 1 0 1 0 1 1]
        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(torch.Tensor(logits)).numpy()

        predictions.extend(probs)
        true_labels.extend(label_ids)

    # multi_label_metrics 함수는 멀티레이블 평가 지표 계산을 합니다.
    # probs = [0.23, 0.91, 0.40, 0.73, 0.10, 0.62, 0.33, 0.77, 0.57] 이라고 가정하였을 때,
    # 여기서 임계값(예: 0.5)을 넘는 확률 값을 가진 레이블은 해당 샘플에 존재한다고 예측합니다.
    # 위 예시에서는 레이블 2, 4, 6, 8, 9가 임계값을 초과하므로, 모델의 예측 레이블은 [2, 4, 6, 8, 9]가 됩니다.
    eval_metrics = multi_label_metrics(np.array(predictions), np.array(true_labels))

    avg_eval_loss = total_eval_loss / len(validation_dataloader)

    return avg_eval_loss, eval_metrics

In [ ]:
# 최소 검증 손실 초기화
min_val_loss = float('inf')

# 메인 학습 & 평가 루프
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))

    # 학습 단계
    train_epoch(model, train_dataloader, optimizer, device)

    print("\nRunning Validation...")
    # 검증 단계
    avg_val_loss, eval_metrics = evaluate(model, validation_dataloader, device)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
    print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
    print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
    print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

    # 검증 손실이 현재까지의 최소값보다 작은 경우 체크포인트 저장
    if avg_val_loss < min_val_loss:
        print(f"Validation loss decreased ({min_val_loss:.2f} --> {avg_val_loss:.2f}).  Saving model ...")
        # 베스트 모델 저장
        torch.save(model.state_dict(), 'model_checkpoint.pt')
        # 최소 검증 손실 업데이트
        min_val_loss = avg_val_loss

======== Epoch 1 / 4 ========


Training Batch: 100%|██████████| 1235/1235 [07:49<00:00,  2.63it/s]



Running Validation...
  Validation Loss: 0.11
  Accuracy: 0.79
  F1 Macro: 0.70
  F1 Micro: 0.84
  F1 Weighted: 0.83
Validation loss decreased (inf --> 0.11).  Saving model ...
======== Epoch 2 / 4 ========


Training Batch: 100%|██████████| 1235/1235 [07:50<00:00,  2.63it/s]



Running Validation...
  Validation Loss: 0.10
  Accuracy: 0.80
  F1 Macro: 0.71
  F1 Micro: 0.85
  F1 Weighted: 0.84
Validation loss decreased (0.11 --> 0.10).  Saving model ...
======== Epoch 3 / 4 ========


Training Batch: 100%|██████████| 1235/1235 [07:50<00:00,  2.62it/s]



Running Validation...
  Validation Loss: 0.10
  Accuracy: 0.79
  F1 Macro: 0.74
  F1 Micro: 0.85
  F1 Weighted: 0.85
======== Epoch 4 / 4 ========


Training Batch: 100%|██████████| 1235/1235 [07:50<00:00,  2.62it/s]



Running Validation...
  Validation Loss: 0.11
  Accuracy: 0.80
  F1 Macro: 0.75
  F1 Micro: 0.85
  F1 Weighted: 0.85


## 모델 저장과 로드

In [ ]:
%pwd

'/content'

In [ ]:
%ls -al

total 432224
drwxr-xr-x 1 root root      4096 Feb 14 12:21 ./
drwxr-xr-x 1 root root      4096 Feb 14 11:47 ../
drwxr-xr-x 4 root root      4096 Feb 12 14:21 .config/
-rw-r--r-- 1 root root 442580501 Feb 14 13:11 model_checkpoint.pt
drwxr-xr-x 1 root root      4096 Feb 12 14:22 sample_data/


In [ ]:
# 베스트 모델 로드
model.load_state_dict(torch.load("model_checkpoint.pt"))

<All keys matched successfully>

# 테스트 데이터에 대한 평가

In [ ]:
avg_val_loss, eval_metrics = evaluate(model, test_dataloader, device)
print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

  Test Loss: 0.10
  Accuracy: 0.80
  F1 Macro: 0.72
  F1 Micro: 0.85
  F1 Weighted: 0.85


# 예측

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512,
                return_all_scores=True, function_to_apply='sigmoid')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
result = pipe('틀니들은 왜 그렇게 민폐를 끼치냐? 특히 나이 먹은 남자들이 심하다')
print(result)

[[{'label': 'LABEL_0', 'score': 0.015342021360993385}, {'label': 'LABEL_1', 'score': 0.0297575443983078}, {'label': 'LABEL_2', 'score': 0.016898056492209435}, {'label': 'LABEL_3', 'score': 0.011106269434094429}, {'label': 'LABEL_4', 'score': 0.9612632393836975}, {'label': 'LABEL_5', 'score': 0.6780894994735718}, {'label': 'LABEL_6', 'score': 0.00563792372122407}, {'label': 'LABEL_7', 'score': 0.0076055387035012245}, {'label': 'LABEL_8', 'score': 0.020267454907298088}]]


In [ ]:
label_dict = {'LABEL_0' : '출신차별', 'LABEL_1' : '외모차별', 'LABEL_2' : '정치성향차별', \
              'LABEL_3': '혐오욕설', 'LABEL_4': '연령차별', 'LABEL_5': '성차별', 'LABEL_6' : '인종차별', \
              'LABEL_7': '종교차별', 'LABEL_8': '해당사항없음'}

In [ ]:
def prediction(text):
  result = pipe(text)
  return [label_dict[res['label']] for res in result[0] if res['score'] > 0.5]

In [ ]:
prediction('틀니들은 왜 그렇게 민폐를 끼치냐? 특히 나이 먹은 남자들이 심하다')

['연령차별', '성차별']